# Iteration 2

In [10]:
!pip install -q -U transformers bitsandbytes

In [11]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import re

In [12]:
DEBUG_LIMIT = None
MODEL_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507"
INPUT_FILE = "zno.test.jsonl"
OUTPUT_FILE = "submission.csv"

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    trust_remote_code=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

In [14]:
data = []
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

In [ ]:
if DEBUG_LIMIT:
    data = data[:DEBUG_LIMIT]
    print(f"Running only first {DEBUG_LIMIT} samples!")

In [16]:
def format_prompt(item):
    question = item['question']
    options_text = ""
    valid_markers = []

    for ans in item['answers']:
        options_text += f"{ans['marker']}) {ans['text']}\n"
        valid_markers.append(ans['marker'])

    messages = [
        {
            "role": "system",
            "content": "Ви — екзаменатор. Ваше завдання — визначити правильну відповідь. Виведіть ТІЛЬКИ одну букву (А, Б, В, Г або Д) без пояснень і без крапок."
        },
        {
            "role": "user",
            "content": f"{question}\n\nВаріанти:\n{options_text}\nВідповідь:"
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text, valid_markers

In [ ]:
results = []
print(f"Starting Fast Inference on {len(data)} items...")

for i, item in enumerate(tqdm(data)):
    prompt, valid_markers = format_prompt(item)
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)

    predicted_letter = "А"

    json_match = re.search(r'{\s*"answer"\s*:\s*"([АБВГД])"\s*}', response, re.IGNORECASE)
    if json_match:
        predicted_letter = json_match.group(1).upper()
    else:
        clean_response = re.sub(r"[^А-Яа-яA-Za-z]", "", response)
        candidates = re.findall(r'\b([АБВГД])\b', response.upper())
        if candidates:
             for cand in reversed(candidates):
                if cand in valid_markers:
                    predicted_letter = cand
                    break
        elif clean_response in valid_markers:
            predicted_letter = clean_response

    results.append({"id": item['id'], "correct_answers": predicted_letter})

    if i < 3:
        print(f"\n[DEBUG ID {item['id']}]")
        print(f"Output: {response}")
        print(f"Parsed: {predicted_letter}")


🚀 Starting Fast Inference on 751 items...


  0%|          | 1/751 [00:05<1:04:06,  5.13s/it]


[DEBUG ID 0]
Output: Б
Parsed: Б


  0%|          | 2/751 [00:09<1:01:11,  4.90s/it]


[DEBUG ID 1]
Output: В
Parsed: В


  0%|          | 3/751 [00:15<1:03:30,  5.09s/it]


[DEBUG ID 2]
Output: А
Parsed: А


  1%|          | 4/751 [00:20<1:05:21,  5.25s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df.to_csv(OUTPUT_FILE, index=False)
print(f" Finished! Saved to {OUTPUT_FILE}")
print(df.head())